## Visualize outputs


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys,inspect
sys.path.insert(0,"..")

import matplotlib.pyplot as plt
from matplotlib import rc

macos = True
if macos == True:
    rc('font',**{'family':'sans-serif','sans-serif':['Computer Modern Roman']})
    rc('text', usetex=True)

# Font Size
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 30}

import cv2
import numpy as np
import string
import random

In [3]:
# compare-tryons-all -> visualize tryon images of different clothing for each configuration
#

def visualize(idx, **images):
    """Plot images in one row.""" 
    n = len(images)
    fig = plt.figure(figsize=(60, 40))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        #if idx==0:
        plt.title(' '.join(name.split('_')).lower(), fontsize=40)
        if i ==0:
            w,h = (1,25)
            fs = 1.0
            color = (0,0,0)
            #color = (255,255,255)
            font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
            cv2.putText(image, str(idx), (w,h), font, fs, color, 1, cv2.LINE_AA)
        plt.imshow(image, cmap='gray')
        plt.axis("off")
        #plt.tight_layout()
    plt.savefig("../outputs/vis/compare-warprefs/{}.png".format(idx), facecolor="white", bbox_inches = 'tight')
    plt.show()
    
    
def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    f = dir.split('/')[-1].split('_')[-1]
    #print (dir, f)
    dirs= os.listdir(dir)
    for img in dirs:

        path = os.path.join(dir, img)
        #print(path)
        images.append(path)
    return images

def read_image(path):
    image = cv2.imread(path, -1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def read_image_(path):
    image = cv2.imread(path, -1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (192, 256))
    return image

In [4]:
!rm ../outputs/robustness_random/acgpn_gen/.DS_Store
!rm ../outputs/robustness_random/fifa_gen/.DS_Store

rm: cannot remove '../outputs/robustness_random/acgpn_gen/.DS_Store': No such file or directory
rm: cannot remove '../outputs/robustness_random/fifa_gen/.DS_Store': No such file or directory


In [ ]:
# Path to model outputs
algo1 = "cpvtonpp"
algo2 = "acgpn_paper"
algo3 = "with_res"
algo4 = "with_all_res"
algo5 = "with_res_msssim"
algo6 = "final"

## New
algo7 = decavton_fifa_wo_fab":

# Path to output tryon images
#mode = "all" # all, all_same, easy, medium, hard
#mode = "bpmask_pred"
mode = "warped_clothes_ref"

if mode == "all":
    algo1_tryon_path = "../outputs/{}/{}/".format(algo1, mode) # only all
    algo1_tryon_files = sorted(make_dataset(algo1_tryon_path))
else:
    pass

algo2_tryon_path = "../outputs/{}/{}/".format(algo2, mode)
algo3_tryon_path = "../outputs/{}/{}/".format(algo3, mode)
algo4_tryon_path = "../outputs/{}/{}/".format(algo4, mode)
algo5_tryon_path = "../outputs/{}/{}/".format(algo5, mode)
algo6_tryon_path = "../outputs/{}/{}/".format(algo6, mode)

# File paths to tryon images
#algo1_tryon_files = sorted(make_dataset(algo1_tryon_path))
algo2_tryon_files = sorted(make_dataset(algo2_tryon_path))
algo3_tryon_files = sorted(make_dataset(algo3_tryon_path))
algo4_tryon_files = sorted(make_dataset(algo4_tryon_path))
algo5_tryon_files = sorted(make_dataset(algo5_tryon_path))
algo6_tryon_files = sorted(make_dataset(algo6_tryon_path))

# Path to reference person and cloth images
ref_person_path = "../../datasets/acgpn_data/try_on_testing/"

persons = []
clothes = []

# for all
with open(os.path.join("../../datasets/acgpn_data/try_on_testing/test_pairs.txt"), 'r') as f:
    for line in f.readlines():
        h_name, c_name = line.strip().split()
        persons.append(h_name)
        clothes.append(c_name)


ref_person_paths = [os.path.join(ref_person_path, "test_img", x) for x in persons]
target_clothes_paths = [os.path.join(ref_person_path, "test_color", x) for x in clothes]

# GT segs
gt_seg = "../outputs/{}/{}/".format("acgpn_paper", "bpmask")
gt_seg_files = sorted(make_dataset(gt_seg))



## Robustness test
cs_1 = []
ps_1 = []
vtons_1 = []
wrs_1 = []
ws_1 = []

vtons_2 = []
wrs_2 = []
ws_2 = []

r_fifa = "../outputs/robustness_random/{}/".format("acgpn_gen") # acgpn
idxs = sorted(os.listdir(r_fifa))
for idx in idxs:
    files = sorted(make_dataset(os.path.join(r_fifa, idx)))
    for f in files:
        if "person" in f:
            ps_1.append(f)
        if "cloth" in f:
            cs_1.append(f)
        if "tryon" in f:
            vtons_1.append(f)
        if "_refined" in f:
            wrs_1.append(f)
        if "warped." in f:
            ws_1.append(f)

In [ ]:
!rm ../outputs/robustness_random/acgpn/.DS_Store
!rm ../outputs/robustness_random/fifa/.DS_Store

In [ ]:
r_fifa = "../outputs/robustness_random/{}/".format("fifa_gen") # fifa
idxs = sorted(os.listdir(r_fifa))
for idx in idxs:
    files = sorted(make_dataset(os.path.join(r_fifa, idx)))
    for f in files:
        if "tryon" in f:
            vtons_2.append(f)
        if "_refined" in f:
            wrs_2.append(f)
        if "warped." in f:
            ws_2.append(f)

### Show all images

In [ ]:
# Using this....
# i = 0
# for num in range(3):
#     visualize(i, reference_person=read_image(ref_person_paths[num]), target_clothes=read_image(target_clothes_paths[num]),
#               cpvtonpp=read_image(algo1_tryon_files[num]),
#               acgpn=read_image(algo2_tryon_files[num]),
#               with_res=read_image(algo3_tryon_files[num]),
#               with_all_res=read_image(algo4_tryon_files[num]),
#               with_res_msssim=read_image(algo5_tryon_files[num]),
#               Ours=read_image(algo6_tryon_files[num]))
#     i+=1


# i = 0
# for num in range(3):
#     visualize(i, reference_person=read_image(ref_person_paths[num]), target_clothes=read_image(target_clothes_paths[num]),
#               cpvtonpp=read_image(algo1_tryon_files[num]),
#               acgpn=read_image(algo2_tryon_files[num]),
#               with_res=read_image(algo3_tryon_files[num]),
#               with_all_res=read_image(algo4_tryon_files[num]),
#               with_res_msssim=read_image(algo5_tryon_files[num]),
#               Ours=read_image(algo6_tryon_files[num]))
#     i+=1


# bpmasks
# i = 0
# for num in range(500):
#     visualize(i, reference_person=read_image(ref_person_paths[num]), target_clothes=read_image(target_clothes_paths[num]),
#               acgpn=read_image(algo2_tryon_files[num]),
#               with_res=read_image(algo3_tryon_files[num]))
#     i+=1

# warped refined clothes
# i = 0
# for num in range(2032):
#     visualize(i, target_clothes=read_image(target_clothes_paths[num]),
#               wo_inp=read_image(algo3_tryon_files[num]),
#               w_inp=read_image(algo6_tryon_files[num]),
#              reference_person=read_image(ref_person_paths[num]))
#     i+=1

In [ ]:
# Analysis:
# good_idxs = [4,12,13,22,28,42,89,95,104,110,
#            121,125,126,128,137,168,172,187,
#            221,222,251,300,315,318,342,350,383,
#           422,439,636,660,
#           737,827,919,1012,1024,1025,
#           1048,1164,1181,1419,1600,1663,
#           1723,1746,1790,1941,1942,1949,
#           1961,1970,1980,1983,1986,1988]

# tryon idxs
good_idxs = [4,12,13,22,28,42,95,104,110,
           125,128,168,172,187,
           221,222,251,342,350,383,
          439,636,660,
          737,827,919,1012,1024,1025,
          1048,1164,1181,1419,1600,1663,
          1723,1746,1790,1941,1942,1949
          ,1970,1980,1988]

In [ ]:
len(good_idxs)

In [ ]:
#complex pose, 
#texture of clothing, 
#embroidery of clothing, 
#retains collar shape of clothing

# tryon idxs
nums = [95,104,342,
       187,1048,1941,
       128,827,636,
       439,660,1949]

In [ ]:
nums

In [ ]:
len(nums)

In [ ]:
for x in nums:
    print(ref_person_paths[x])

### Plot to compare try-ons with SOTA

In [ ]:
rows = int(len(nums) / 3)
cols = 15
fig, axarr = plt.subplots(rows, cols, figsize=(40, 15), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(rp1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc1=read_image(target_clothes_paths[nums[v+r]])
    a1=read_image(algo1_tryon_files[nums[v+r]])
    cpv1 = read_image(algo2_tryon_files[nums[v+r]])
    o1=read_image(algo6_tryon_files[nums[v+r]])
    
    rp2=read_image(ref_person_paths[nums[v+r+1]])
    cv2.putText(rp2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc2=read_image(target_clothes_paths[nums[v+r+1]])
    a2=read_image(algo1_tryon_files[nums[v+r+1]])
    cpv2 = read_image(algo2_tryon_files[nums[v+r+1]])
    o2=read_image(algo6_tryon_files[nums[v+r+1]])
    
    rp3=read_image(ref_person_paths[nums[v+r+2]])
    cv2.putText(rp3, str(alphabet_list[v+r+2]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc3=read_image(target_clothes_paths[nums[v+r+2]])
    a3=read_image(algo1_tryon_files[nums[v+r+2]])
    cpv3 = read_image(algo2_tryon_files[nums[v+r+2]])
    o3=read_image(algo6_tryon_files[nums[v+r+2]])
    
    v+=2
    
    images = [rp1, tc1, a1, cpv1, o1, rp2, tc2, a2, cpv2, o2, rp3, tc3, a3, cpv3, o3]
    
    captions = ["Reference \n Person", "Target \n Clothes", "CP-VTON+", "ACGPN", "FIFA (Ours)",
                "Reference \n Person", "Target \n Clothes", "CP-VTON+", "ACGPN", "FIFA (Ours)",
                "Reference \n Person", "Target \n Clothes", "CP-VTON+", "ACGPN", "FIFA (Ours)"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=40)

plt.savefig("../outputs/compare_sota.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
# nums = [4,12,22,28,110,121, 125,168,221,251,300,350,383,737
#         ,919,1012,1024,1025,1164,1600, 1633,
#         1723,1746,1790,1942,1970, 1980, 1983, 1986, 1988][:16]


# Fig 1
# fails to preserve texture and embroidery of target clothing -> 1988,350,383,737,1790,919,121,1025
# bad in complex pose -> 4, 22, 1024, 1164, 1746
# blurry -> 1600, 1970, 1980

# Fig 2
# unrealistic human skin parts -> 12, 110, 251
# keeps old clothing -> 28, 125, 1942, 300
# human color -> 1012, 1723
# front and black clothing mixed -> 1986, 168, 221,1663,1983


#nums = [1988,350,383,737,1790,919,121,1025, 4, 22, 1024, 1164, 1746, 1600, 1970, 1980]
nums = [12, 110, 251, 28, 125, 1942, 300, 1012, 1723, 1986, 168, 221,1663,1983]

len(nums)

## Extensive results

In [ ]:
rows = int(len(nums)/ 2)
cols = 12
fig, axarr = plt.subplots(rows, cols, figsize=(30, 24), constrained_layout=True) # 24 or 27


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(rp1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc1=read_image(target_clothes_paths[nums[v+r]])
    a1=read_image(algo1_tryon_files[nums[v+r]])
    cpv1 = read_image(algo2_tryon_files[nums[v+r]])
    o0=read_image(algo5_tryon_files[nums[v+r]])
    o1=read_image(algo6_tryon_files[nums[v+r]])
    
    rp2=read_image(ref_person_paths[nums[v+r+1]])
    cv2.putText(rp2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc2=read_image(target_clothes_paths[nums[v+r+1]])
    a2=read_image(algo1_tryon_files[nums[v+r+1]])
    cpv2 = read_image(algo2_tryon_files[nums[v+r+1]])
    o00=read_image(algo5_tryon_files[nums[v+r+1]])
    o2=read_image(algo6_tryon_files[nums[v+r+1]])
    
    v+=1
    
    images = [rp1, tc1, a1, cpv1, o0, o1, rp2, tc2, a2, cpv2, o00, o2]
    
    captions = ["Reference \n Person", "Target \n Clothes", "CP-VTON+", "ACGPN", "FIFA (Ours) \n (w/o MCM)", "FIFA (Ours)",
               "Reference \n Person", "Target \n Clothes", "CP-VTON+", "ACGPN", "FIFA (Ours) \n (w/o MCM)", "FIFA (Ours)"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=30)

plt.savefig("../outputs/compare_sota_extensive2.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

## Robustness test

In [ ]:
# cs_1 = []
# ps_1 = []
# vtons_1 = []
# wrs_1 = []
# ws_1 = []

# vtons_2 = []
# wrs_2 = []
# ws_2 = []

In [ ]:
nums = [x for x in range(10)]
random.shuffle(nums)
nums

In [ ]:
rows = int(len(nums)/ 2)
cols = 8
fig, axarr = plt.subplots(rows, cols, figsize=(30, 26), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image_(ps_1[nums[v+r]])
    tc1=read_image_(cs_1[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(tc1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    
    a1=read_image_(vtons_1[nums[v+r]])
    cpv1 = read_image_(vtons_2[nums[v+r]])
    
    rp2=read_image_(ps_1[nums[v+r+1]])
    tc2=read_image_(cs_1[nums[v+r+1]])
    cv2.putText(tc2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    a2=read_image_(vtons_1[nums[v+r+1]])
    cpv2 = read_image_(vtons_2[nums[v+r+1]])
    
    v+=1
    
    images = [tc1, rp1, a1, cpv1, tc2, rp2, a2, cpv2]
    
    captions = ["Target \n Clothes", "Reference \n Person", "ACGPN", "FIFA (Ours)",
               "Target \n Clothes", "Reference \n Person", "ACGPN", "FIFA (Ours)",]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=45)

plt.savefig("../outputs/robustness_tryons_random_viton.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

### Plot for ablation 1 (ablation for res, loss, inpaint)

In [ ]:
nums = [1181, 1419]

# res blocks -> RBs 
# multi-scale strutural constraint -> MSS
# Masked Clothing Modeling -> MCM objective

In [ ]:
rows = int(len(nums))
cols = 7
fig, axarr = plt.subplots(rows, cols, figsize=(37, 15), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    tc1=read_image(target_clothes_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(tc1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    a1=read_image(algo1_tryon_files[nums[v+r]])
    cpv1 = read_image(algo2_tryon_files[nums[v+r]])
    o1=read_image(algo3_tryon_files[nums[v+r]])
    o2=read_image(algo5_tryon_files[nums[v+r]])
    o3=read_image(algo6_tryon_files[nums[v+r]])
    
    images = [tc1, rp1, a1, cpv1, o1, o2, o3]
    
    captions = ["Target \n Clothes", "Reference \n Person", "CP-VTON+", "ACGPN", "FIFA \n w/ RBs ",
                "FIFA \n w/ RBs + MSC", "FIFA \n w/ RBs + MSC + MCM"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=40)

plt.savefig("../outputs/ablation_1.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

### Plot for abl 2 (w/o and with res)

mode should be "bpmask_pred"

In [ ]:
nums = [39, 57, 133]

In [ ]:
rows = int(len(nums))
cols = 4
fig, axarr = plt.subplots(rows, cols, figsize=(20, 20), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    tc1=read_image(target_clothes_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(tc1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    a1 = read_image(algo2_tryon_files[nums[v+r]])
    a2=read_image(algo3_tryon_files[nums[v+r]])
    
    images = [tc1, rp1, a1, a2]
    
    captions = ["Target \n Clothes", "Reference \n Person", " FIFA (Ours) \n w/o RBs", " FIFA (Ours) \n w/ RBs"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=30)

plt.savefig("../outputs/ablation_2_res.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

### Plot for ablation 3 (w/o and w inpaint)

In [ ]:
# preserve logo a,b
# complex pose c
# synthesize cloth in non-target body parts a,b, d

In [ ]:
nums = [938, 998, 357, 59] 

In [ ]:
rows = int(len(nums)/ 2)
cols = 8
fig, axarr = plt.subplots(rows, cols, figsize=(27, 10), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    tc1=read_image(target_clothes_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(tc1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    
    a1=read_image(algo3_tryon_files[nums[v+r]])
    cpv1 = read_image(algo6_tryon_files[nums[v+r]])
    
    rp2=read_image(ref_person_paths[nums[v+r+1]])
    tc2=read_image(target_clothes_paths[nums[v+r+1]])
    cv2.putText(tc2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    a2=read_image(algo3_tryon_files[nums[v+r+1]])
    cpv2 = read_image(algo6_tryon_files[nums[v+r+1]])
    
    v+=1
    
    images = [tc1, rp1, a1, cpv1, tc2, rp2, a2, cpv2]
    
    captions = ["Target \n Clothes", "Reference \n Person", "FIFA (Ours) \n (w/o MCM)", "FIFA (Ours) \n (w/ MCM)",
               "Target \n Clothes", "Reference \n Person", "FIFA (Ours) \n (w/o MCM)", "FIFA (Ours) \n (w/ MCM)"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=35)

plt.savefig("../outputs/abl_compare_warps.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

### Plot extensive warps

In [ ]:
#nums= [5,11,23,59,66,68,80,218,243,294,315,381,414,512,522,652,684,864,874,1000][:14]
nums= [11,23,59,66,68,294,5,315,381,512,684,864,874,1000]
len(nums)

In [ ]:
rows = int(len(nums)/ 2)
cols = 8
fig, axarr = plt.subplots(rows, cols, figsize=(26, 31), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    tc1=read_image(target_clothes_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(tc1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    
    a1=read_image(algo3_tryon_files[nums[v+r]])
    cpv1 = read_image(algo6_tryon_files[nums[v+r]])
    
    rp2=read_image(ref_person_paths[nums[v+r+1]])
    tc2=read_image(target_clothes_paths[nums[v+r+1]])
    cv2.putText(tc2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    a2=read_image(algo3_tryon_files[nums[v+r+1]])
    cpv2 = read_image(algo6_tryon_files[nums[v+r+1]])
    
    v+=1
    
    images = [tc1, rp1, a1, cpv1, tc2, rp2, a2, cpv2]
    
    captions = ["Reference \n Person", "Target \n Clothes", "FIFA (Ours) \n (w/o MCM)", "FIFA (Ours) \n (w/ MCM)",
               "Reference \n Person", "Target \n Clothes", "FIFA (Ours) \n (w/o MCM)", "FIFA (Ours) \n (w/ MCM)"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=30)

plt.savefig("../outputs/warps_ext.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

# OLD

## Vis warped clothes

In [ ]:
# i = 0
# for num in range(5):
#     visualize(i, target_cloth=read_image(cloth_files[num]), warped_by_stn=read_image(wcr_files[num]),
#               warp_refine=read_image(wcr_ref_files[num]),
#              reference_person=read_image(ref_person_paths[num]))
#     i+=1

In [ ]:
cloth_path = "../outputs/{}/{}/".format(algo1, "clothes")
wcr_ref_path1 = "../outputs/{}/{}/".format(algo1, "warped_clothes_ref")
wcr_ref_path2 = "../outputs/{}/{}/".format(algo2, "warped_clothes_ref")

cloth_files = sorted(make_dataset(cloth_path))
wcr1 = sorted(make_dataset(wcr_ref_path1))
wcr2 = sorted(make_dataset(wcr_ref_path2))

In [ ]:
i = 0
for num in range(2032):
    visualize(i, target_cloth=read_image(cloth_files[num]), resunet=read_image(wcr1[num]),
              ssl=read_image(wcr2[num]))
    i+=1

### Vis seg masks

In [ ]:
gt_seg = "../outputs/{}/{}/".format("segs", "bpmask")
a1_seg = "../outputs/{}/{}/".format("segs", "bpmask_pred_unet")
a2_seg = "../outputs/{}/{}/".format("segs", "bpmask_pred")

gt_seg_files = sorted(make_dataset(gt_seg))
a1_seg_files = sorted(make_dataset(a1_seg))
a2_seg_files = sorted(make_dataset(a2_seg))

In [ ]:
i = 0

nums = [0, 2, 6, 34, 69, 510, 853]
for num in nums:
    visualize(i, ground_seg=read_image(gt_seg_files[num]), unet_seg=read_image(a1_seg_files[num]),
              resunet_seg=read_image(a2_seg_files[num]))
    i+=1

In [ ]:
nums = [0, 2, 6, 34, 69, 510] # 853

In [ ]:
rows = int(len(nums) / 3)
cols = 12
fig, axarr = plt.subplots(rows, cols, figsize=(42, 10), constrained_layout=True)

alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(rp1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc1=read_image(gt_seg_files[nums[v+r]])
    a1_seg=read_image(a1_seg_files[nums[v+r]])
    a2_seg = read_image(a2_seg_files[nums[v+r]])
    
    rp2=read_image(ref_person_paths[nums[v+r+1]])
    cv2.putText(rp2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc2=read_image(gt_seg_files[nums[v+r+1]])
    a21_seg=read_image(a1_seg_files[nums[v+r+1]])
    a22_seg = read_image(a2_seg_files[nums[v+r+1]])
    
    rp3=read_image(ref_person_paths[nums[v+r+2]])
    cv2.putText(rp3, str(alphabet_list[v+r+2]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc3=read_image(gt_seg_files[nums[v+r+2]])
    a23_seg=read_image(a1_seg_files[nums[v+r+2]])
    a223_seg = read_image(a2_seg_files[nums[v+r+2]])
    
    v+=2
    
    images = [rp1, tc1, a1_seg, a2_seg, rp2, tc2, a21_seg, a22_seg, rp3, tc3, a23_seg, a223_seg]
    
    
    captions = ["Reference \n Person", "Ground Truth \n Body Part Mask", "w/o \n residual units", "with \n residual units",
                "Reference \n Person", "Ground Truth \n Body Part Mask", "w/o \nresidual units", "with \n residual units",
               "Reference \n Person", "Ground Truth \n Body Part Mask", "w/o  \n residual units", "with  \n residual units"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal')
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=35)

plt.savefig("../outputs/vis_mask.png", facecolor="white", bbox_inches = 'tight', dpi=300)

### Show masks

In [ ]:
# nums = np.array([6, 82, 17, 18, 23, 28, 57, 59, 67, 71, 120, 169, 239, 305, 307, 507, 1868, 1961, 1981, 2012])[-12:]
# nums

In [ ]:
# i = 0
# for num in nums:
#     visualize(i, reference_person=read_image(ref_person_paths[num]),
#               Clothing_Region_GT=read_image(gt_wcr_files[num])[:,:,0], 
#               Unet=read_image(algo1_wcr_path[num])[:,:,0], resunet=read_image(algo2_wcr_path[num])[:,:,0])
#     i+=1

In [ ]:
# Plot using npstack

# arrs = []
# for refp, clt, gtwcr, a1wcr, a2wcr, a1, a2 in zip(ref_person_paths[:slc], target_clothes_paths[:3], 
#                   gt_wcr_files[:slc], algo1_wcr_path[:slc], algo2_wcr_path[:slc], algo1_tryon_files[:slc],
#                   algo2_tryon_files[:slc]):
    
#     person=read_image(refp)
#     cloth=read_image(clt)

#     wcr=read_image(gtwcr) 
#     #wcr= np.where(wcr==127, 0, wcr)

#     wcr_acgpn=read_image(a1wcr) 
#     #wcr_acgpn= np.where(wcr_acgpn==127, 0, wcr_acgpn)

#     wcr_ours=read_image(a2wcr)
#     #wcr_ours= np.where(wcr_ours==127, 0, wcr_ours)

#     acgpn=read_image(a1)
#     ours=read_image(a2)
    
#     out = np.hstack((person, cloth, wcr, wcr_acgpn, wcr_ours, acgpn, ours))
#     #out = np.pad(out, pad_width=1, mode='constant', constant_values=0)
#     arrs.append(out)    

In [ ]:
# final_output = np.vstack((x for x in arrs))
# final_output.shape

In [ ]:
# plt.figure(figsize=(18, 16))
# plt.axis("off")
# plt.imshow(final_output, cmap='gray');

### Plot images with mask

In [ ]:
# nums = np.random.randint(0, 2032, 5)
# nums

In [ ]:
# nums = [i for i in range(5)] # 5

In [ ]:
# rows = len(nums)
# cols = 7
# fig, axarr = plt.subplots(rows, cols, figsize=(40, 38), constrained_layout=True)

# for r in range(rows):
#     reference_person=read_image(ref_person_paths[nums[r]])
#     target_clothes=read_image(target_clothes_paths[nums[r]])
#     Clothing_Region_GT=read_image(gt_wcr_files[nums[r]])[:,:,0]
#     Unet=read_image(algo1_wcr_path[nums[r]])[:,:,0]
#     resunet=read_image(algo2_wcr_path[nums[r]])[:,:,0]
#     ACGPN=read_image(algo1_tryon_files[nums[r]])
#     Ours=read_image(algo2_tryon_files[nums[r]])
    
#     images = [reference_person, target_clothes, Clothing_Region_GT, Unet, resunet, ACGPN, Ours]
#     captions = ["Reference \n Person", "Target \n Clothes", "Clothing Region \n Ground Truth", "U-Net", "Res U-Net", "ACGPN", "Ours"]
    
#     for c in range(cols):
#         axarr[r, c].imshow(images[c], cmap='gray')
#         axarr[r, c].axis("off")
#         axarr[r, c].set_aspect('equal')
#         if r==0:
#             axarr[r, c].set_title(captions[c], fontsize=50)

# plt.savefig("../outputs/visualization_all.png", facecolor="white", bbox_inches = 'tight', dpi=300)

## Tryon images

In [ ]:
nums = np.random.randint(0, 2032, 12)
nums

In [ ]:
# acgpn and res acgpn
#nums = np.array([6, 82, 17, 18, 23, 28, 57, 59, 67, 71, 120, 169, 239, 305, 307, 507, 1868, 1961, 1981, 2012])[-12:]

# With acgpn, cpvton+, res acgpn
#nums = np.array([5,6,9,18,59,172,189,190,275,277,322,566,743,1024,1417,1954,2012, 2031])[:12]
nums = np.array([2031,2012,9,18,743,1417,189,190,275,277,322,566,743,1024,1417,1954,2012, 2031])[:12]

In [ ]:
len(nums)

In [ ]:
# nums = [i for i in range(12)]
# nums

In [ ]:
rows = int(len(nums) / 3)
cols = 15
fig, axarr = plt.subplots(rows, cols, figsize=(40, 15), constrained_layout=True)


alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string)

v = 0
for r in range(rows):
    rp1=read_image(ref_person_paths[nums[v+r]])
    w,h = (1,25)
    fs = 1.0
    color = (0,0,0)
    font = cv2.FONT_HERSHEY_SIMPLEX #FONT_HERSHEY_DUPLEX  #press tab for different operations
    cv2.putText(rp1, str(alphabet_list[v+r]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc1=read_image(target_clothes_paths[nums[v+r]])
    a1=read_image(algo1_tryon_files[nums[v+r]])
    cpv1 = read_image(algo3_tryon_files[nums[v+r]])
    o1=read_image(algo2_tryon_files[nums[v+r]])
    
    rp2=read_image(ref_person_paths[nums[v+r+1]])
    cv2.putText(rp2, str(alphabet_list[v+r+1]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc2=read_image(target_clothes_paths[nums[v+r+1]])
    a2=read_image(algo1_tryon_files[nums[v+r+1]])
    cpv2 = read_image(algo3_tryon_files[nums[v+r+1]])
    o2=read_image(algo2_tryon_files[nums[v+r+1]])
    
    rp3=read_image(ref_person_paths[nums[v+r+2]])
    cv2.putText(rp3, str(alphabet_list[v+r+2]), (w,h), font, fs, color, 1, cv2.LINE_AA)
    tc3=read_image(target_clothes_paths[nums[v+r+2]])
    a3=read_image(algo1_tryon_files[nums[v+r+2]])
    cpv3 = read_image(algo3_tryon_files[nums[v+r+2]])
    o3=read_image(algo2_tryon_files[nums[v+r+2]])
    
    v+=2
    
    images = [rp1, tc1, a1, cpv1, o1, rp2, tc2, a2, cpv2, o2, rp3, tc3, a3, cpv3, o3]
    
    captions = ["Reference \n Person", "Target \n Clothes", "ACGPN", "CP-VTON+", "Ours",
                "Reference \n Person", "Target \n Clothes", "ACGPN", "CP-VTON+", "Ours",
                "Reference \n Person", "Target \n Clothes", "ACGPN", "CP-VTON+", "Ours"]
    
    for c in range(cols):
        axarr[r, c].imshow(images[c], cmap='gray')
        axarr[r, c].axis("off")
        axarr[r, c].set_aspect('equal') 
        if r==0:
            axarr[r, c].set_title(captions[c], fontsize=40)

plt.savefig("../outputs/visualization_tryons.png", facecolor="white", bbox_inches = 'tight', dpi=300)